In [1]:
# Define helper function for loading data
import pandas as pd
from sklearn.model_selection import train_test_split

def data_loader(csv_file_path, labelling_criteria, dataset_name, sample_size, num_examples):
    combined_data = pd.read_csv(csv_file_path, usecols=['text', 'label', 'group'])

    label2id = {label: (1 if label == labelling_criteria else 0) for label in combined_data['label'].unique()}
    combined_data['label'] = combined_data['label'].map(label2id)

    combined_data['data_name'] = dataset_name

    if sample_size >= len(combined_data):
        sampled_data = combined_data
    else:
        sample_proportion = sample_size / len(combined_data)
        sampled_data, _ = train_test_split(combined_data, train_size=sample_proportion, stratify=combined_data['label'],
                                           random_state=42)

    train_data, test_data = train_test_split(sampled_data, test_size=0.2, random_state=42,
                                             stratify=sampled_data['label'])

    print("First few examples from the training data:")
    print(train_data.head(num_examples))
    print("First few examples from the testing data:")
    print(test_data.head(num_examples))
    print("Train data size:", len(train_data))
    print("Test data size:", len(test_data))

    return train_data, test_data

In [2]:
# Define helper function for merging data
def merge_datasets(train_data_candidate, test_data_candidate, train_data_established, test_data_established, num_examples):
    merged_train_data = pd.concat([train_data_candidate, train_data_established], ignore_index=True)
    merged_test_data = pd.concat([test_data_candidate, test_data_established], ignore_index=True)

    print("First few examples from merged training data:")
    print(merged_train_data.head(num_examples))
    print("First few examples from merged testing data:")
    print(merged_test_data.head(num_examples))
    print("Train data merged size:", len(merged_train_data))
    print("Test data merged size:", len(merged_test_data))

    return merged_train_data, merged_test_data

In [3]:
# Define function for fine tuning language model
import os
import numpy as np
import logging
from sklearn.metrics import classification_report, precision_recall_fscore_support, balanced_accuracy_score
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, pipeline
from codecarbon import EmissionsTracker

In [13]:
# Enable progress bar and set up logging
os.environ["HUGGINGFACE_TRAINER_ENABLE_PROGRESS_BAR"] = "1"
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.INFO)
def train_model(train_data, model_path, batch_size, epoch, learning_rate, model_output_base_dir, dataset_name, seed):

    np.random.seed(seed)
    num_labels = len(train_data['label'].unique())
    print(f"Number of unique labels: {num_labels}")

    tracker = EmissionsTracker()
    tracker.start()

    model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=num_labels, ignore_mismatched_sizes=True)
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    if model_path.startswith("gpt"):
        tokenizer.pad_token = tokenizer.eos_token
        model.config.pad_token_id = tokenizer.pad_token_id

    def tokenize_function(examples):
        return tokenizer(examples["text"], padding=True, truncation=True, max_length=128) #change from 512 to 128

    train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)
    tokenized_train = Dataset.from_pandas(train_data).map(tokenize_function, batched=True).map(lambda examples: {'labels': examples['label']})
    print("Sample tokenized input from train:", tokenized_train[0])
    tokenized_val = Dataset.from_pandas(val_data).map(tokenize_function, batched=True).map(lambda examples: {'labels': examples['label']})
    print("Sample tokenized input from validation:", tokenized_train[0])

    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)
        precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='macro')
        balanced_acc = balanced_accuracy_score(labels, predictions)
        return {"precision": precision, "recall": recall, "f1": f1, "balanced accuracy": balanced_acc}

    model_output_dir = os.path.join(model_output_base_dir, dataset_name)
    os.makedirs(model_output_dir, exist_ok=True)

    training_args = TrainingArguments(
        output_dir=model_output_dir, num_train_epochs=epoch, evaluation_strategy="epoch", learning_rate=learning_rate,
        per_device_train_batch_size=batch_size, per_device_eval_batch_size=batch_size, weight_decay=0.01,
        save_strategy="epoch", load_best_model_at_end=True, save_total_limit=1)

    trainer = Trainer(
        model=model, args=training_args, tokenizer=tokenizer, train_dataset=tokenized_train,
        eval_dataset=tokenized_val, compute_metrics=compute_metrics)

    trainer.train()
    trainer.save_model(model_output_dir)

    emissions = tracker.stop()
    print(f"Estimated total emissions: {emissions} kg CO2")

    return model_output_dir

In [14]:
# Define function for evaluating the model
def evaluate_model(test_data, model_output_dir, result_output_base_dir, dataset_name, seed):

    np.random.seed(seed)
    num_labels = len(test_data['label'].unique())
    print(f"Number of unique labels: {num_labels}")

    model = AutoModelForSequenceClassification.from_pretrained(model_output_dir, num_labels=num_labels,
                                                               ignore_mismatched_sizes=True)
    tokenizer = AutoTokenizer.from_pretrained(model_output_dir)

    if model_output_dir.startswith("gpt"):
        tokenizer.pad_token = tokenizer.eos_token
        model.config.pad_token_id = tokenizer.pad_token_id

    def tokenize_function(examples):
        return tokenizer(examples["text"], padding=True, truncation=True, max_length=128)

    tokenized_test = Dataset.from_pandas(test_data).map(tokenize_function, batched=True).map(
        lambda examples: {'labels': examples['label']})
    print("Sample tokenized input from test:", tokenized_test[0])

    result_output_dir = os.path.join(result_output_base_dir, dataset_name)
    os.makedirs(result_output_dir, exist_ok=True)

    pipe = pipeline("text-classification", model= model,tokenizer=tokenizer,device=-1)

    predictions = pipe(test_data['text'].to_list(), return_all_scores=True)
    # 这些红线是 Pylance 误把 x 当成 Tensor，认为不能用字符串当索引 导致的静态类型错误，不影响实际运行。你可以通过类型注解、# type: ignore 或改写成显式循环来消掉它们。
    pred_labels = [int(max(pred, key=lambda x: x['score'])['label'].split('_')[-1]) for pred in predictions]
    pred_probs = [max(pred, key=lambda x: x['score'])['score'] for pred in predictions]
    y_true = test_data['label'].tolist()

    results_df = pd.DataFrame({
        'text': test_data['text'],
        'predicted_label': pred_labels,
        'predicted_probability': pred_probs,
        'actual_label': y_true,
        'group': test_data['group'],
        'dataset_name': test_data['data_name']
    })

    results_file_path = os.path.join(result_output_dir, "full_results.csv")
    results_df.to_csv(results_file_path, index=False)

    report = classification_report(y_true,pred_labels,output_dict=True)
    df_report = pd.DataFrame(report).transpose()
    result_file_path = os.path.join(result_output_dir, "classification_report.csv")
    df_report.to_csv(result_file_path)

    return df_report

In [15]:
# Load and combine relevant datasets
sample_size = 5000
train_data_winoqueer_gpt_augmentation, test_data_winoqueer_gpt_augmentation = data_loader(
    csv_file_path='Winoqueer - GPT Augmentation.csv', 
    labelling_criteria='stereotype', 
    dataset_name='Winoqueer - GPT Augmentation', 
    sample_size=sample_size, 
    num_examples=5)
train_data_seegull_gpt_augmentation, test_data_seegull_gpt_augmentation = data_loader(
    csv_file_path='SeeGULL - GPT Augmentation.csv', 
    labelling_criteria='stereotype', 
    dataset_name='SeeGULL - GPT Augmentation', 
    sample_size=sample_size, 
    num_examples=5)
train_data_mgsd, test_data_mgsd = data_loader(
    csv_file_path='MGSD.csv', 
    labelling_criteria='stereotype', 
    dataset_name='MGSD', 
    sample_size=sample_size, 
    num_examples=5)
train_data_merged_winoqueer_gpt_augmentation, test_data_merged_winoqueer_gpt_augmentation = merge_datasets(train_data_candidate = train_data_winoqueer_gpt_augmentation, test_data_candidate = test_data_winoqueer_gpt_augmentation, train_data_established = train_data_mgsd, test_data_established = test_data_mgsd, num_examples=5)
train_data_merged_seegull_gpt_augmentation, test_data_merged_seegull_gpt_augmentation = merge_datasets(train_data_candidate = train_data_seegull_gpt_augmentation, test_data_candidate = test_data_seegull_gpt_augmentation, train_data_established = train_data_mgsd, test_data_established = test_data_mgsd, num_examples=5)
train_data_merged_winoqueer_seegull_gpt_augmentation, test_data_merged_winoqueer_seegull_gpt_augmentation = merge_datasets(train_data_candidate = train_data_seegull_gpt_augmentation, test_data_candidate = test_data_seegull_gpt_augmentation, train_data_established = train_data_merged_winoqueer_gpt_augmentation, test_data_established = test_data_merged_winoqueer_gpt_augmentation, num_examples=5)

First few examples from the training data:
                                                   text  label        group  \
68                            Gay people are survivors.      1          Gay   
2269                      Transgender people are doors.      0  Transgender   
3208  most people who are Pansexual are undermined i...      0    Pansexual   
826                           most lesbians are sinful.      1      Lesbian   
2452   All LGBTQ people are unfit for spaceship travel.      0        LGBTQ   

                         data_name  
68    Winoqueer - GPT Augmentation  
2269  Winoqueer - GPT Augmentation  
3208  Winoqueer - GPT Augmentation  
826   Winoqueer - GPT Augmentation  
2452  Winoqueer - GPT Augmentation  
First few examples from the testing data:
                                                   text  label        group  \
3236                  most people who are Gay are kites      0          Gay   
2749            we expect that LGBTQ people are paints.     

In [ ]:
train_model(train_data_merged_winoqueer_seegull_gpt_augmentation, 
    model_path='albert/albert-base-v2', 
    batch_size=16, # from 64 to 16
    epoch=1, 
    learning_rate=2e-5, 
    model_output_base_dir='model_output_albertv2', 
    dataset_name='merged_winoqueer_seegull_gpt_augmentation_trained', 
    seed=42)

[codecarbon INFO @ 09:25:42] [setup] RAM Tracking...
[codecarbon INFO @ 09:25:42] [setup] GPU Tracking...
[codecarbon INFO @ 09:25:42] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 09:25:42] [setup] CPU Tracking...
[codecarbon WARNING @ 09:25:42] No CPU tracking mode found. Falling back on CPU constant mode. 
 Windows OS detected: Please install Intel Power Gadget to measure CPU



Number of unique labels: 2


[codecarbon INFO @ 09:25:44] CPU Model on constant consumption mode: Intel(R) Core(TM) i7-10875H CPU @ 2.30GHz
[codecarbon INFO @ 09:25:44] >>> Tracker's metadata:
[codecarbon INFO @ 09:25:44]   Platform system: Windows-10-10.0.26100-SP0
[codecarbon INFO @ 09:25:44]   Python version: 3.10.19
[codecarbon INFO @ 09:25:44]   CodeCarbon version: 2.8.0
[codecarbon INFO @ 09:25:44]   Available RAM : 15.789 GB
[codecarbon INFO @ 09:25:44]   CPU count: 16
[codecarbon INFO @ 09:25:44]   CPU model: Intel(R) Core(TM) i7-10875H CPU @ 2.30GHz
[codecarbon INFO @ 09:25:44]   GPU count: 1
[codecarbon INFO @ 09:25:44]   GPU model: 1 x NVIDIA GeForce RTX 2060
[codecarbon INFO @ 09:25:47] Saving emissions data to file c:\Users\15509\Desktop\UK courses\AI for SD\Coursework 2\HEARTS-Text-Stereotype-Detection-main\Model Training and Evaluation\emissions.csv
loading configuration file config.json from cache at C:\Users\15509\.cache\huggingface\hub\models--albert--albert-base-v2\snapshots\8e2f239c5f8a2c0f2537

Map:   0%|          | 0/6613 [00:00<?, ? examples/s]

Map:   0%|          | 0/6613 [00:00<?, ? examples/s]

...

[codecarbon INFO @ 09:42:03] Energy consumed for RAM : 0.001603 kWh. RAM Power : 5.920974254608154 W
[codecarbon INFO @ 09:42:03] Energy consumed for all GPUs : 0.001747 kWh. Total GPU Power : 6.445976025908097 W
[codecarbon INFO @ 09:42:03] Energy consumed for all CPUs : 0.006095 kWh. Total CPU Power : 22.5 W
[codecarbon INFO @ 09:42:03] 0.009445 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:42:18] Energy consumed for RAM : 0.001628 kWh. RAM Power : 5.920974254608154 W
[codecarbon INFO @ 09:42:18] Energy consumed for all GPUs : 0.001774 kWh. Total GPU Power : 6.39551424337787 W
[codecarbon INFO @ 09:42:18] Energy consumed for all CPUs : 0.006189 kWh. Total CPU Power : 22.5 W
[codecarbon INFO @ 09:42:18] 0.009590 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:42:33] Energy consumed for RAM : 0.001652 kWh. RAM Power : 5.920974254608154 W
[codecarbon INFO @ 09:42:33] Energy consumed for all GPUs : 0.001800 kWh. Total GPU Power : 6.371289010052819 

{'eval_loss': 0.3995645344257355, 'eval_precision': 0.8061402082129911, 'eval_recall': 0.7684906016116206, 'eval_f1': 0.7816456135962535, 'eval_balanced accuracy': 0.7684906016116206, 'eval_runtime': 57.3863, 'eval_samples_per_second': 28.822, 'eval_steps_per_second': 1.812, 'epoch': 1.0}




Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from model_output_albertv2\merged_winoqueer_seegull_gpt_augmentation_trained\checkpoint-414 (score: 0.3995645344257355).
[codecarbon WARNING @ 09:42:45] Another instance of codecarbon is already running. Exiting.
Saving model checkpoint to model_output_albertv2\merged_winoqueer_seegull_gpt_augmentation_trained
Configuration saved in model_output_albertv2\merged_winoqueer_seegull_gpt_augmentation_trained\config.json
Model weights saved in model_output_albertv2\merged_winoqueer_seegull_gpt_augmentation_trained\model.safetensors
tokenizer config file saved in model_output_albertv2\merged_winoqueer_seegull_gpt_augmentation_trained\tokenizer_config.json
Special tokens file saved in model_output_albertv2\merged_winoqueer_seegull_gpt_augmentation_trained\special_tokens_map.json
[codecarbon INFO @ 09:42:46] Energy consumed for RAM : 0.001673 kWh. RAM Power : 5.920974254608154 W
[codecarbon 

{'train_runtime': 1015.4997, 'train_samples_per_second': 6.512, 'train_steps_per_second': 0.408, 'train_loss': 0.48747024904702596, 'epoch': 1.0}
Estimated total emissions: 0.00234238601529457 kg CO2


'model_output_albertv2\\merged_winoqueer_seegull_gpt_augmentation_trained'

In [ ]:
train_model(train_data_merged_winoqueer_seegull_gpt_augmentation, 
    model_path='model_output_albertv2_2epoches/merged_winoqueer_seegull_gpt_augmentation_trained', 
    batch_size=16, # from 64 to 16
    epoch=1, 
    learning_rate=2e-5, 
    model_output_base_dir='model_output_albertv2_2epoches', 
    dataset_name='merged_winoqueer_seegull_gpt_augmentation_trained', 
    seed=42)

[codecarbon INFO @ 03:20:41] [setup] RAM Tracking...
[codecarbon INFO @ 03:20:41] [setup] GPU Tracking...


Number of unique labels: 2


[codecarbon INFO @ 03:20:42] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 03:20:42] [setup] CPU Tracking...
[codecarbon WARNING @ 03:20:42] No CPU tracking mode found. Falling back on CPU constant mode. 
 Windows OS detected: Please install Intel Power Gadget to measure CPU

[codecarbon INFO @ 03:20:44] CPU Model on constant consumption mode: Intel(R) Core(TM) i7-10875H CPU @ 2.30GHz
[codecarbon INFO @ 03:20:44] >>> Tracker's metadata:
[codecarbon INFO @ 03:20:44]   Platform system: Windows-10-10.0.26100-SP0
[codecarbon INFO @ 03:20:44]   Python version: 3.10.19
[codecarbon INFO @ 03:20:44]   CodeCarbon version: 2.8.0
[codecarbon INFO @ 03:20:44]   Available RAM : 15.789 GB
[codecarbon INFO @ 03:20:44]   CPU count: 16
[codecarbon INFO @ 03:20:44]   CPU model: Intel(R) Core(TM) i7-10875H CPU @ 2.30GHz
[codecarbon INFO @ 03:20:44]   GPU count: 1
[codecarbon INFO @ 03:20:44]   GPU model: 1 x NVIDIA GeForce RTX 2060
[codecarbon INFO @ 03:20:47] Saving emissions data to file c:\Users\1

Map:   0%|          | 0/6613 [00:00<?, ? examples/s]

Map:   0%|          | 0/6613 [00:00<?, ? examples/s]

...

Configuration saved in model_output_albertv2_2epoches\merged_winoqueer_seegull_gpt_augmentation_trained\checkpoint-414\config.json
Model weights saved in model_output_albertv2_2epoches\merged_winoqueer_seegull_gpt_augmentation_trained\checkpoint-414\model.safetensors
tokenizer config file saved in model_output_albertv2_2epoches\merged_winoqueer_seegull_gpt_augmentation_trained\checkpoint-414\tokenizer_config.json
Special tokens file saved in model_output_albertv2_2epoches\merged_winoqueer_seegull_gpt_augmentation_trained\checkpoint-414\special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from model_output_albertv2_2epoches\merged_winoqueer_seegull_gpt_augmentation_trained\checkpoint-414 (score: 0.35729289054870605).
[codecarbon WARNING @ 03:37:33] Another instance of codecarbon is already running. Exiting.
Saving model checkpoint to model_output_albertv2_2epoches\merged_winoqueer_seegull_gpt_augmentation_traine

{'train_runtime': 1002.6741, 'train_samples_per_second': 6.595, 'train_steps_per_second': 0.413, 'train_loss': 0.3570411424130057, 'epoch': 1.0}


[codecarbon INFO @ 03:37:34] Energy consumed for all GPUs : 0.000000 kWh. Total GPU Power : 0.05883265248076508 W
[codecarbon INFO @ 03:37:34] Energy consumed for all CPUs : 0.006288 kWh. Total CPU Power : 22.5 W
[codecarbon INFO @ 03:37:34] 0.007829 kWh of electricity used since the beginning.


Estimated total emissions: 0.0018600549026269544 kg CO2


'model_output_albertv2_2epoches\\merged_winoqueer_seegull_gpt_augmentation_trained'

In [18]:
'model_output_albertv2_2epoches/merged_winoqueer_seegull_gpt_augmentation_trained'
evaluate_model(test_data_mgsd, model_output_dir='model_output_albertv2_2epoches/merged_winoqueer_seegull_gpt_augmentation_trained', 
               result_output_base_dir='result_output_albertv2_2epoches/merged_winoqueer_seegull_gpt_augmentation_trained', 
               dataset_name='mgsd', 
               seed=42)

loading configuration file model_output_albertv2_2epoches/merged_winoqueer_seegull_gpt_augmentation_trained\config.json
Model config AlbertConfig {
  "_name_or_path": "model_output_albertv2_2epoches/merged_winoqueer_seegull_gpt_augmentation_trained",
  "architectures": [
    "AlbertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu_new",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 0,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dt

Number of unique labels: 2


loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Sample tokenized input from test: {'group': 'race', 'text': 'We have learned many things about Yemen in school. Some people are so kind.', 'label': 0, 'data_name': 'MGSD', '__index_level_0__': 43942, 'input_ids': [2, 95, 57, 2691, 151, 564, 88, 12911, 19, 116, 9, 109, 148, 50, 86, 825, 9, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': 0}


c:\Users\15509\anaconda3\envs\hearts\lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


,precision,recall,f1-score,support
0,0.793578,0.788754,0.791159,658.000
1,0.598266,0.605263,0.601744,342.000
accuracy,0.726000,0.726000,0.726000,0.726
macro avg,0.695922,0.697008,0.696451,1000.000
weighted avg,0.726781,0.726000,0.726379,1000.000


In [17]:
evaluate_model(test_data_mgsd, model_output_dir='model_output_albertv2/merged_winoqueer_seegull_gpt_augmentation_trained', 
               result_output_base_dir='result_output_albertv2/merged_winoqueer_seegull_gpt_augmentation_trained', 
               dataset_name='mgsd', 
               seed=42)

loading configuration file model_output_albertv2/merged_winoqueer_seegull_gpt_augmentation_trained\config.json
Model config AlbertConfig {
  "_name_or_path": "model_output_albertv2/merged_winoqueer_seegull_gpt_augmentation_trained",
  "architectures": [
    "AlbertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu_new",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 0,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
 

Number of unique labels: 2


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Sample tokenized input from test: {'group': 'race', 'text': 'We have learned many things about Yemen in school. Some people are so kind.', 'label': 0, 'data_name': 'MGSD', '__index_level_0__': 43942, 'input_ids': [2, 95, 57, 2691, 151, 564, 88, 12911, 19, 116, 9, 109, 148, 50, 86, 825, 9, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': 0}


c:\Users\15509\anaconda3\envs\hearts\lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


,precision,recall,f1-score,support
0,0.740940,0.838906,0.786885,658.000
1,0.584314,0.435673,0.499162,342.000
accuracy,0.701000,0.701000,0.701000,0.701
macro avg,0.662627,0.637289,0.643024,1000.000
weighted avg,0.687374,0.701000,0.688484,1000.000


In [19]:
evaluate_model(test_data_merged_winoqueer_seegull_gpt_augmentation, 
               model_output_dir='model_output_albertv2_2epoches/merged_winoqueer_seegull_gpt_augmentation_trained', 
               result_output_base_dir='result_output_albertv2_2epoches/merged_winoqueer_seegull_gpt_augmentation_trained', 
               dataset_name='merged_winoqueer_seegull_gpt_augmentation', 
               seed=42)

loading configuration file model_output_albertv2_2epoches/merged_winoqueer_seegull_gpt_augmentation_trained\config.json
Model config AlbertConfig {
  "_name_or_path": "model_output_albertv2_2epoches/merged_winoqueer_seegull_gpt_augmentation_trained",
  "architectures": [
    "AlbertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu_new",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 0,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dt

Number of unique labels: 2


Map:   0%|          | 0/2067 [00:00<?, ? examples/s]

Map:   0%|          | 0/2067 [00:00<?, ? examples/s]

Sample tokenized input from test: {'text': 'The Somali refugees were labeled as unacquainted by some', 'label': 0, 'group': 'Somalis', 'data_name': 'SeeGULL - GPT Augmentation', 'input_ids': [2, 14, 16330, 8790, 46, 14348, 28, 367, 26692, 108, 1427, 34, 109, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': 0}


c:\Users\15509\anaconda3\envs\hearts\lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


,precision,recall,f1-score,support
0,0.879913,0.883126,0.881517,1369.000000
1,0.769120,0.763610,0.766355,698.000000
accuracy,0.842767,0.842767,0.842767,0.842767
macro avg,0.824516,0.823368,0.823936,2067.000000
weighted avg,0.842499,0.842767,0.842628,2067.000000


In [20]:
evaluate_model(test_data_seegull_gpt_augmentation, 
               model_output_dir='model_output_albertv2_2epoches/merged_winoqueer_seegull_gpt_augmentation_trained', 
               result_output_base_dir='result_output_albertv2_2epoches/merged_winoqueer_seegull_gpt_augmentation_trained', 
               dataset_name='seegull_gpt_augmentation', seed=42)

loading configuration file model_output_albertv2_2epoches/merged_winoqueer_seegull_gpt_augmentation_trained\config.json
Model config AlbertConfig {
  "_name_or_path": "model_output_albertv2_2epoches/merged_winoqueer_seegull_gpt_augmentation_trained",
  "architectures": [
    "AlbertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu_new",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 0,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dt

Number of unique labels: 2


Map:   0%|          | 0/414 [00:00<?, ? examples/s]

Map:   0%|          | 0/414 [00:00<?, ? examples/s]

Sample tokenized input from test: {'text': 'The Somali refugees were labeled as unacquainted by some', 'label': 0, 'group': 'Somalis', 'data_name': 'SeeGULL - GPT Augmentation', '__index_level_0__': 840, 'input_ids': [2, 14, 16330, 8790, 46, 14348, 28, 367, 26692, 108, 1427, 34, 109, 3, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0], 'labels': 0}


c:\Users\15509\anaconda3\envs\hearts\lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


,precision,recall,f1-score,support
0,0.922535,0.949275,0.935714,276.000000
1,0.892308,0.840580,0.865672,138.000000
accuracy,0.913043,0.913043,0.913043,0.913043
macro avg,0.907421,0.894928,0.900693,414.000000
weighted avg,0.912459,0.913043,0.912367,414.000000


In [21]:
evaluate_model(test_data_winoqueer_gpt_augmentation, 
               model_output_dir='model_output_albertv2_2epoches/merged_winoqueer_seegull_gpt_augmentation_trained', 
               result_output_base_dir='result_output_albertv2_2epoches/merged_winoqueer_seegull_gpt_augmentation_trained', 
               dataset_name='seegull_gpt_augmentation', seed=42)

loading configuration file model_output_albertv2_2epoches/merged_winoqueer_seegull_gpt_augmentation_trained\config.json
Model config AlbertConfig {
  "_name_or_path": "model_output_albertv2_2epoches/merged_winoqueer_seegull_gpt_augmentation_trained",
  "architectures": [
    "AlbertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu_new",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 0,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dt

Number of unique labels: 2


Map:   0%|          | 0/653 [00:00<?, ? examples/s]

Map:   0%|          | 0/653 [00:00<?, ? examples/s]

Sample tokenized input from test: {'text': 'most people who are Gay are kites', 'label': 0, 'group': 'Gay', 'data_name': 'Winoqueer - GPT Augmentation', '__index_level_0__': 3236, 'input_ids': [2, 127, 148, 72, 50, 3398, 50, 19196, 18, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': 0}


c:\Users\15509\anaconda3\envs\hearts\lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


,precision,recall,f1-score,support
0,0.981651,0.983908,0.982778,435.000000
1,0.967742,0.963303,0.965517,218.000000
accuracy,0.977029,0.977029,0.977029,0.977029
macro avg,0.974697,0.973605,0.974148,653.000000
weighted avg,0.977008,0.977029,0.977016,653.000000


In [ ]:
evaluate_model(test_data_merged_winoqueer_seegull_gpt_augmentation, 
               model_output_dir='model_output_albertv2/merged_winoqueer_seegull_gpt_augmentation_trained', 
               result_output_base_dir='result_output_albertv2/merged_winoqueer_seegull_gpt_augmentation_trained', 
               dataset_name='merged_winoqueer_seegull_gpt_augmentation', 
               seed=42)

loading configuration file model_output_albertv2/merged_winoqueer_seegull_gpt_augmentation_trained\config.json
Model config AlbertConfig {
  "_name_or_path": "model_output_albertv2/merged_winoqueer_seegull_gpt_augmentation_trained",
  "architectures": [
    "AlbertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu_new",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 0,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
 

Number of unique labels: 2


Map:   0%|          | 0/2067 [00:00<?, ? examples/s]

Map:   0%|          | 0/2067 [00:00<?, ? examples/s]

c:\Users\15509\anaconda3\envs\hearts\lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
Disabling tokenizer parallelism, we're using DataLoader multithreading already


Sample tokenized input from test: {'text': 'The Somali refugees were labeled as unacquainted by some', 'label': 0, 'group': 'Somalis', 'data_name': 'SeeGULL - GPT Augmentation', 'input_ids': [2, 14, 16330, 8790, 46, 14348, 28, 367, 26692, 108, 1427, 34, 109, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': 0}


,precision,recall,f1-score,support
0,0.828877,0.905771,0.865620,1369.00000
1,0.774081,0.633238,0.696612,698.00000
accuracy,0.813740,0.813740,0.813740,0.81374
macro avg,0.801479,0.769504,0.781116,2067.00000
weighted avg,0.810373,0.813740,0.808548,2067.00000


In [ ]:
evaluate_model(test_data_seegull_gpt_augmentation, 
               model_output_dir='model_output_albertv2/merged_winoqueer_seegull_gpt_augmentation_trained', 
               result_output_base_dir='result_output_albertv2/merged_winoqueer_seegull_gpt_augmentation_trained', 
               dataset_name='seegull_gpt_augmentation', seed=42)


loading configuration file model_output_albertv2/merged_winoqueer_seegull_gpt_augmentation_trained\config.json
Model config AlbertConfig {
  "_name_or_path": "model_output_albertv2/merged_winoqueer_seegull_gpt_augmentation_trained",
  "architectures": [
    "AlbertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu_new",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 0,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
 

Number of unique labels: 2


Map:   0%|          | 0/414 [00:00<?, ? examples/s]

Map:   0%|          | 0/414 [00:00<?, ? examples/s]

Sample tokenized input from test: {'text': 'The Somali refugees were labeled as unacquainted by some', 'label': 0, 'group': 'Somalis', 'data_name': 'SeeGULL - GPT Augmentation', '__index_level_0__': 840, 'input_ids': [2, 14, 16330, 8790, 46, 14348, 28, 367, 26692, 108, 1427, 34, 109, 3, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0], 'labels': 0}


,precision,recall,f1-score,support
0,0.910345,0.956522,0.932862,276.000000
1,0.903226,0.811594,0.854962,138.000000
accuracy,0.908213,0.908213,0.908213,0.908213
macro avg,0.906785,0.884058,0.893912,414.000000
weighted avg,0.907972,0.908213,0.906895,414.000000


In [11]:
evaluate_model(test_data_winoqueer_gpt_augmentation, 
               model_output_dir='model_output_albertv2/merged_winoqueer_seegull_gpt_augmentation_trained', 
               result_output_base_dir='result_output_albertv2/merged_winoqueer_seegull_gpt_augmentation_trained', 
               dataset_name='winoqueer_gpt_augmentation', seed=42)

loading configuration file model_output_albertv2/merged_winoqueer_seegull_gpt_augmentation_trained\config.json
Model config AlbertConfig {
  "_name_or_path": "model_output_albertv2/merged_winoqueer_seegull_gpt_augmentation_trained",
  "architectures": [
    "AlbertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu_new",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 0,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
 

Number of unique labels: 2


Map:   0%|          | 0/653 [00:00<?, ? examples/s]

Map:   0%|          | 0/653 [00:00<?, ? examples/s]

Sample tokenized input from test: {'text': 'most people who are Gay are kites', 'label': 0, 'group': 'Gay', 'data_name': 'Winoqueer - GPT Augmentation', '__index_level_0__': 3236, 'input_ids': [2, 127, 148, 72, 50, 3398, 50, 19196, 18, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': 0}


c:\Users\15509\anaconda3\envs\hearts\lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


,precision,recall,f1-score,support
0,0.927473,0.970115,0.948315,435.000000
1,0.934343,0.848624,0.889423,218.000000
accuracy,0.929556,0.929556,0.929556,0.929556
macro avg,0.930908,0.909369,0.918869,653.000000
weighted avg,0.929766,0.929556,0.928654,653.000000


In [10]:
evaluate_model(test_data_merged_winoqueer_seegull_gpt_augmentation, 
               model_output_dir='model_output_albertv2/merged_winoqueer_seegull_gpt_augmentation_trained', 
               result_output_base_dir='result_output_albertv2/merged_winoqueer_seegull_gpt_augmentation_trained', 
               dataset_name='merged_winoqueer_seegull_gpt_augmentation', 
               seed=42)

loading configuration file model_output_albertv2/merged_winoqueer_seegull_gpt_augmentation_trained\config.json
Model config AlbertConfig {
  "_name_or_path": "model_output_albertv2/merged_winoqueer_seegull_gpt_augmentation_trained",
  "architectures": [
    "AlbertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu_new",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 0,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
 

Number of unique labels: 2


Map:   0%|          | 0/2067 [00:00<?, ? examples/s]

Map:   0%|          | 0/2067 [00:00<?, ? examples/s]

Sample tokenized input from test: {'text': 'The Somali refugees were labeled as unacquainted by some', 'label': 0, 'group': 'Somalis', 'data_name': 'SeeGULL - GPT Augmentation', 'input_ids': [2, 14, 16330, 8790, 46, 14348, 28, 367, 26692, 108, 1427, 34, 109, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': 0}


c:\Users\15509\anaconda3\envs\hearts\lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


,precision,recall,f1-score,support
0,0.830872,0.904310,0.866037,1369.000000
1,0.772964,0.638968,0.699608,698.000000
accuracy,0.814707,0.814707,0.814707,0.814707
macro avg,0.801918,0.771639,0.782822,2067.000000
weighted avg,0.811317,0.814707,0.809836,2067.000000


In [ ]:
train_model(train_data_merged_winoqueer_seegull_gpt_augmentation, 
    model_path='xlm-roberta-base', 
    batch_size=16, # from 64 to 16
    epoch=1, 
    learning_rate=2e-5, 
    model_output_base_dir='model_output_xlm-roberta-base', 
    dataset_name='merged_winoqueer_seegull_gpt_augmentation_trained', 
    seed=42)

[codecarbon INFO @ 08:46:03] [setup] RAM Tracking...
[codecarbon INFO @ 08:46:03] [setup] GPU Tracking...
[codecarbon INFO @ 08:46:03] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 08:46:03] [setup] CPU Tracking...
[codecarbon WARNING @ 08:46:03] No CPU tracking mode found. Falling back on CPU constant mode. 
 Windows OS detected: Please install Intel Power Gadget to measure CPU



Number of unique labels: 2


[codecarbon INFO @ 08:46:05] CPU Model on constant consumption mode: Intel(R) Core(TM) i7-10875H CPU @ 2.30GHz
[codecarbon INFO @ 08:46:05] >>> Tracker's metadata:
[codecarbon INFO @ 08:46:05]   Platform system: Windows-10-10.0.26100-SP0
[codecarbon INFO @ 08:46:05]   Python version: 3.10.19
[codecarbon INFO @ 08:46:05]   CodeCarbon version: 2.8.0
[codecarbon INFO @ 08:46:05]   Available RAM : 15.789 GB
[codecarbon INFO @ 08:46:05]   CPU count: 16
[codecarbon INFO @ 08:46:05]   CPU model: Intel(R) Core(TM) i7-10875H CPU @ 2.30GHz
[codecarbon INFO @ 08:46:05]   GPU count: 1
[codecarbon INFO @ 08:46:05]   GPU model: 1 x NVIDIA GeForce RTX 2060
[codecarbon INFO @ 08:46:08] Saving emissions data to file c:\Users\15509\Desktop\UK courses\AI for SD\Coursework 2\HEARTS-Text-Stereotype-Detection-main\Model Training and Evaluation\emissions.csv


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

c:\Users\15509\anaconda3\envs\hearts\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\15509\.cache\huggingface\hub\models--xlm-roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
loading configuration file config.json from cache at C:\Users\15509\.cache\huggingface\hub\models--xlm-roberta-base\

...

Model weights saved in model_output_albertv2\merged_winoqueer_seegull_gpt_augmentation_trained\checkpoint-414\model.safetensors
[codecarbon INFO @ 09:18:24] Energy consumed for RAM : 0.003182 kWh. RAM Power : 5.920974254608154 W
[codecarbon INFO @ 09:18:24] Energy consumed for all GPUs : 0.003447 kWh. Total GPU Power : 6.357348749607534 W
tokenizer config file saved in model_output_albertv2\merged_winoqueer_seegull_gpt_augmentation_trained\checkpoint-414\tokenizer_config.json
[codecarbon INFO @ 09:18:24] Energy consumed for all CPUs : 0.012101 kWh. Total CPU Power : 22.5 W
[codecarbon INFO @ 09:18:24] 0.018730 kWh of electricity used since the beginning.
Special tokens file saved in model_output_albertv2\merged_winoqueer_seegull_gpt_augmentation_trained\checkpoint-414\special_tokens_map.json
[codecarbon INFO @ 09:18:24] 0.002302 g.CO2eq/s mean an estimation of 72.58202896567883 kg.CO2eq/year
[codecarbon INFO @ 09:18:39] Energy consumed for RAM : 0.003207 kWh. RAM Power : 5.920974254608

{'train_runtime': 1646.8074, 'train_samples_per_second': 4.016, 'train_steps_per_second': 0.251, 'train_loss': 0.543231945682839, 'epoch': 1.0}


Model weights saved in model_output_albertv2\merged_winoqueer_seegull_gpt_augmentation_trained\model.safetensors
[codecarbon INFO @ 09:19:22] Energy consumed for RAM : 0.003278 kWh. RAM Power : 5.920974254608154 W
[codecarbon INFO @ 09:19:22] Energy consumed for all GPUs : 0.003544 kWh. Total GPU Power : 5.975447951952986 W
[codecarbon INFO @ 09:19:22] Energy consumed for all CPUs : 0.012463 kWh. Total CPU Power : 22.5 W
[codecarbon INFO @ 09:19:22] 0.019285 kWh of electricity used since the beginning.
tokenizer config file saved in model_output_albertv2\merged_winoqueer_seegull_gpt_augmentation_trained\tokenizer_config.json
Special tokens file saved in model_output_albertv2\merged_winoqueer_seegull_gpt_augmentation_trained\special_tokens_map.json
[codecarbon INFO @ 09:19:22] Energy consumed for RAM : 0.003278 kWh. RAM Power : 5.920974254608154 W
[codecarbon INFO @ 09:19:22] Energy consumed for all GPUs : 0.003544 kWh. Total GPU Power : 2.6820229055687137 W
[codecarbon INFO @ 09:19:22]

Estimated total emissions: 0.00458208868225132 kg CO2


'model_output_albertv2\\merged_winoqueer_seegull_gpt_augmentation_trained'

In [14]:
evaluate_model(test_data_merged_winoqueer_seegull_gpt_augmentation, 
               model_output_dir='model_output_xlm-roberta-base/merged_winoqueer_seegull_gpt_augmentation_trained', 
               result_output_base_dir='result_output_xlm-roberta-base/merged_winoqueer_seegull_gpt_augmentation_trained', 
               dataset_name='merged_winoqueer_seegull_gpt_augmentation', 
               seed=42)

loading configuration file model_output_xlm-roberta-base/merged_winoqueer_seegull_gpt_augmentation_trained\config.json
Model config XLMRobertaConfig {
  "_name_or_path": "model_output_xlm-roberta-base/merged_winoqueer_seegull_gpt_augmentation_trained",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.46.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading weights f

Number of unique labels: 2


All model checkpoint weights were used when initializing XLMRobertaForSequenceClassification.

All the weights of XLMRobertaForSequenceClassification were initialized from the model checkpoint at model_output_xlm-roberta-base/merged_winoqueer_seegull_gpt_augmentation_trained.
If your task is similar to the task the model of the checkpoint was trained on, you can already use XLMRobertaForSequenceClassification for predictions without further training.
loading file sentencepiece.bpe.model
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json


Map:   0%|          | 0/2067 [00:00<?, ? examples/s]

Map:   0%|          | 0/2067 [00:00<?, ? examples/s]

c:\Users\15509\anaconda3\envs\hearts\lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
Disabling tokenizer parallelism, we're using DataLoader multithreading already


Sample tokenized input from test: {'text': 'The Somali refugees were labeled as unacquainted by some', 'label': 0, 'group': 'Somalis', 'data_name': 'SeeGULL - GPT Augmentation', 'input_ids': [0, 581, 92422, 211190, 90, 3542, 67967, 297, 237, 51, 47215, 73, 3674, 390, 3060, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': 0}


,precision,recall,f1-score,support
0,0.845546,0.859752,0.852590,1369.000000
1,0.715556,0.691977,0.703569,698.000000
accuracy,0.803096,0.803096,0.803096,0.803096
macro avg,0.780551,0.775864,0.778079,2067.000000
weighted avg,0.801650,0.803096,0.802267,2067.000000


In [4]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer

model_path = 'model_output_xlm-roberta-base/merged_winoqueer_seegull_gpt_augmentation_trained'   # 你的模型目录
pipe = pipeline("text-classification", model=model_path, tokenizer=model_path, return_all_scores=True)
model_path = 'model_output_albertv2/merged_winoqueer_seegull_gpt_augmentation_trained'
pipe_albert = pipeline("text-classification", model=model_path, tokenizer=model_path, return_all_scores=True)



c:\Users\15509\anaconda3\envs\hearts\lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [26]:
text = "这个评论带有刻板印象吗？"
result = pipe(text)
print(result)
result = pipe_albert(text)
print(result)

[[{'label': 'LABEL_0', 'score': 0.40001264214515686}, {'label': 'LABEL_1', 'score': 0.5999873280525208}]]
[[{'label': 'LABEL_0', 'score': 0.8098716139793396}, {'label': 'LABEL_1', 'score': 0.19012843072414398}]]


In [7]:
text = "大学生都很笨"
result = pipe(text)
print(result)
result = pipe_albert(text)
print(result)

[[{'label': 'LABEL_0', 'score': 0.08588053286075592}, {'label': 'LABEL_1', 'score': 0.9141194224357605}]]
[[{'label': 'LABEL_0', 'score': 0.624780535697937}, {'label': 'LABEL_1', 'score': 0.3752194344997406}]]


In [9]:
text = "有很笨的大学生"
result = pipe(text)
print(result)

[[{'label': 'LABEL_0', 'score': 0.13456343114376068}, {'label': 'LABEL_1', 'score': 0.8654365539550781}]]


In [10]:
text = "Stupid undergraduate student exists"
result = pipe_albert(text)
print(result)

[[{'label': 'LABEL_0', 'score': 0.24039381742477417}, {'label': 'LABEL_1', 'score': 0.759606122970581}]]


In [ ]:
text = "大学生は全部馬鹿です"
result = pipe(text)
print(result)

[[{'label': 'LABEL_0', 'score': 0.07690759003162384}, {'label': 'LABEL_1', 'score': 0.9230924248695374}]]


In [8]:

text = "大学生には馬鹿な人もいます"
result = pipe(text)
print(result)

[[{'label': 'LABEL_0', 'score': 0.0859021320939064}, {'label': 'LABEL_1', 'score': 0.914097785949707}]]


In [19]:
text = "中国人都很聪明"
result = pipe(text)
print(result)

[[{'label': 'LABEL_0', 'score': 0.788849413394928}, {'label': 'LABEL_1', 'score': 0.2111506164073944}]]


In [28]:
text = "Chinese are all  smart"
result = pipe(text)
print(result)
result = pipe_albert(text)
print(result)

[[{'label': 'LABEL_0', 'score': 0.7089645862579346}, {'label': 'LABEL_1', 'score': 0.29103535413742065}]]
[[{'label': 'LABEL_0', 'score': 0.8592768907546997}, {'label': 'LABEL_1', 'score': 0.1407231241464615}]]


In [22]:
text = "女人就是不能当法官"
result = pipe(text)
print(result)

[[{'label': 'LABEL_0', 'score': 0.1442098170518875}, {'label': 'LABEL_1', 'score': 0.8557901978492737}]]


In [6]:
sample_size = 300
train_data_mgsd_zh, test_data_mgsd_zh = data_loader(
    csv_file_path='mgsd_zh_dataset.csv', 
    labelling_criteria='stereotype', 
    dataset_name='mgsd_zh', 
    sample_size=sample_size, 
    num_examples=5)

First few examples from the training data:
          group                       text  label data_name
222  profession             那个懒惰的警卫在博物馆工作。      0   mgsd_zh
254        race  我的朋友去了尼泊尔旅行。那里非常贫穷，生活很艰难。      0   mgsd_zh
278      gender                  他是她邪恶的继父。      0   mgsd_zh
90         race     阿拉伯人是一个笼统的称呼。我晚上吃垃圾食品。      0   mgsd_zh
108  profession       那名狱警在监狱里以“像个外星人”而出名。      0   mgsd_zh
First few examples from the testing data:
          group                         text  label data_name
160  profession   老师向学生讲授各门学科。星期二是吃玉米卷饼的好日子。      0   mgsd_zh
220        race  我哥哥学会说葡萄牙语了。贫民窟里的老鼠长得跟猫一样大。      0   mgsd_zh
273  profession    分析师问了一系列具体的问题。这是一个不相关的句子。      0   mgsd_zh
145        race                  这个约旦家庭正在旅行。      0   mgsd_zh
201    religion   我妹妹刚买了一个Brahmin手袋。我们得去买邮票。      0   mgsd_zh
Train data size: 240
Test data size: 60


In [8]:
print(test_data_mgsd_zh)

          group                                               text  label  \
160  profession                         老师向学生讲授各门学科。星期二是吃玉米卷饼的好日子。      0   
220        race                        我哥哥学会说葡萄牙语了。贫民窟里的老鼠长得跟猫一样大。      0   
273  profession                          分析师问了一系列具体的问题。这是一个不相关的句子。      0   
145        race                                        这个约旦家庭正在旅行。      0   
201    religion                         我妹妹刚买了一个Brahmin手袋。我们得去买邮票。      0   
235        race                            苏的女朋友是哥伦比亚人。她喜欢在电脑上查资料。      0   
87         race                                           厄瓜多人是警察。      0   
105        race                         几十年来，荷兰人一直来到这个国家。他们很喜欢中国菜！      0   
257        race                             那名葡萄牙男子走进了大楼。有些人喝牛奶咖啡。      0   
106    religion                                             耶稣是基督徒      0   
149  profession                                   夏令营辅导员给了我一些人生建议。      0   
174        race                                  我遇到过的每一个西班牙人都很勤奋。      0   

In [7]:
evaluate_model(test_data_mgsd_zh, 
               model_output_dir='model_output_xlm-roberta-base/merged_winoqueer_seegull_gpt_augmentation_trained', 
               result_output_base_dir='result_output_xlm-roberta-base/test_data_mgsd_zh', 
               dataset_name='test_data_mgsd_zh', 
               seed=42)

loading configuration file model_output_xlm-roberta-base/merged_winoqueer_seegull_gpt_augmentation_trained\config.json
Model config XLMRobertaConfig {
  "_name_or_path": "model_output_xlm-roberta-base/merged_winoqueer_seegull_gpt_augmentation_trained",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.46.3",
  "type_vo

Number of unique labels: 1


All model checkpoint weights were used when initializing XLMRobertaForSequenceClassification.

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at model_output_xlm-roberta-base/merged_winoqueer_seegull_gpt_augmentation_trained and are newly initialized because the shapes did not match:
- classifier.out_proj.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([1]) in the model instantiated
- classifier.out_proj.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([1, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
loading file sentencepiece.bpe.model
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json


Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

c:\Users\15509\anaconda3\envs\hearts\lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
Disabling tokenizer parallelism, we're using DataLoader multithreading already


Sample tokenized input from test: {'group': 'profession', 'text': '老师向学生讲授各门学科。星期二是吃玉米卷饼的好日子。', 'label': 0, 'data_name': 'mgsd_zh', '__index_level_0__': 160, 'input_ids': [0, 6, 15238, 2110, 7252, 18735, 58016, 4035, 6349, 72584, 30, 32999, 99276, 4011, 199647, 31023, 193950, 33666, 91453, 30, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': 0}


,precision,recall,f1-score,support
0,1.0,1.0,1.0,60.0
accuracy,1.0,1.0,1.0,1.0
macro avg,1.0,1.0,1.0,60.0
weighted avg,1.0,1.0,1.0,60.0


In [10]:
combined_data = pd.read_csv('mgsd_zh_dataset.csv', usecols=['text', 'label', 'group'])

combined_data['data_name'] = 'mgsd_zh_dataset'




In [8]:
print(combined_data)

          group                          text  label        data_name
0          race               印度尼西亚是一个非常干净的国家      0  mgsd_zh_dataset
1          race                   哥伦比亚人性格很平和。      0  mgsd_zh_dataset
2          race                     法国人通常都很多毛      0  mgsd_zh_dataset
3          race             我们遇到的所有英国人都非常有礼貌。      0  mgsd_zh_dataset
4          race    街尾的那个男人是索马里人。他积极为女性争取平等权利。      0  mgsd_zh_dataset
..          ...                           ...    ...              ...
295        race                 有很多关于越南的气球照片。      0  mgsd_zh_dataset
296      gender              她和她的继父有一种书本上的关系。      0  mgsd_zh_dataset
297        race    俄罗斯人不喜欢乌克兰人，不过他们对乌克兰人非常宽容。      0  mgsd_zh_dataset
298  profession  有很多实习生也是研究人员。他们爱评头论足，而且薪水偏低。      0  mgsd_zh_dataset
299        race        那个俄罗斯人放下了他的杯子。里面并没有酒精。      0  mgsd_zh_dataset

[300 rows x 4 columns]


In [11]:
evaluate_model(combined_data, 
               model_output_dir='model_output_xlm-roberta-base/merged_winoqueer_seegull_gpt_augmentation_trained', 
               result_output_base_dir='result_output_xlm-roberta-base/test_data_mgsd_zh', 
               dataset_name='test_data_mgsd_zh', 
               seed=42)

Number of unique labels: 2


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Sample tokenized input from test: {'group': 'race', 'text': '印度尼西亚是一个非常干净的国家', 'label': 0, 'data_name': 'mgsd_zh_dataset', 'input_ids': [0, 6, 26449, 12379, 3891, 10783, 13663, 4528, 135878, 101400, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': 0}


c:\Users\15509\anaconda3\envs\hearts\lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


,precision,recall,f1-score,support
0,0.711538,0.751269,0.730864,197.000000
1,0.467391,0.417476,0.441026,103.000000
accuracy,0.636667,0.636667,0.636667,0.636667
macro avg,0.589465,0.584372,0.585945,300.000000
weighted avg,0.627715,0.636667,0.631353,300.000000
